In [2]:
# !pip install pygame

# REF:
pynput: https://pypi.org/project/pynput/

### For space key recording

In [3]:
# !pip install threading
# !pip install keyboard
# !pip install pynput

In [4]:
import pynput
# pynput.keyboard.Listener.stop()
 

In [2]:
import tkinter as tk
import threading
# from pynput import keyboard
import pynput
import sounddevice as sd
import numpy as np
from queue import Queue
import keyboard as kb



In [3]:
# Global variables
recording = False
audio_data = []
info_label = None  # Declare the label globally
gui_closing = False  # Flag to indicate GUI closing
# info_label = tk.Label(root, text="Press 'Space' to start recording", padx=10, pady=10)
# info_label.pack()
info_queue = Queue()  # Queue for communication between threads
space_event_queue = Queue()  # Queue to store space key events

recording_lock = threading.Lock()

# Event to signal the record thread to stop
stop_record_event = threading.Event()





In [4]:
audio_input_queue = Queue()

In [9]:
stop_record_event.clear()
audio_data = []


def record_audio(audio_data, audio_input_queue):
    # global audio_data
    print("func record_audio")

    #init 
    with audio_input_queue.mutex:
        audio_input_queue.queue.clear()


    while not stop_record_event.is_set():
        event = space_event_queue.get()
        print("record_audio event: ", event)
        if event == "pressed":
            print("Recorder Recording...")
            try:
                event = space_event_queue.get(block=False)
            except:
                event = "pressed"
            while event != "released":
                data = sd.rec(44100, samplerate=44100, channels=1, dtype=np.int16)
                sd.wait()
                audio_input_queue.put(data)
                audio_data.extend(data)
                try:
                    event = space_event_queue.get(block=False)
                except:
                    event = "pressed"
            print("Recorder end Recording...")
    print("end func record_audio")
    print("audio_data: ", len(audio_data))




def on_key_press(key):
    global recording, info_label
    try:
        if key == pynput.keyboard.Key.space:
            with recording_lock:
                if not recording:
                    recording = True
                    # record_audio()
                    space_event_queue.put("pressed")
                    update_label(info_label, "Recording...")  # Update label in main GUI thread
                    # info_queue.put("Recording...")
                    print("Recording...")
        elif key == pynput.keyboard.Key.esc and gui_closing:
            return False  # Stop the listener when 'esc' is detected and GUI is closing
    except Exception as e:
        print(f"Error during key press: {e}")

def on_key_release(key):
    global recording, info_label
    try:
        if key == pynput.keyboard.Key.space:
            with recording_lock:
                if recording == True:
                    recording = False
                    # stop_recording()
                    space_event_queue.put("released")
                    update_label(info_label, "Press 'Space' to start recording")  # Update label in main GUI thread
                    # info_queue.put("Press 'Space' to start recording")
                    print("Press 'Space' to start recording")
    except Exception as e:
        print(f"Error during key release: {e}")


def update_label(label, text):
    label.config(text=text)

# Create listener
listener = pynput.keyboard.Listener(on_press=on_key_press, on_release=on_key_release)

# Keyboard listener thread
# def keyboard_listener_thread():
listener.start()
    # listener.join()  # Wait for the listener to finish


record_thread = threading.Thread(target=record_audio, args=(audio_data, audio_input_queue))
record_thread.start()

# # # GUI thread
# def gui_thread():
def on_closing():
    global gui_closing
    # recording_lock.acquire()
    # if recording:
    #     recording_lock.release()
    #     stop_recording()
    # else:
    #     recording_lock.release()
    gui_closing = True
    stop_record_event.set()  # Signal the record thread to stop
    pynput.keyboard.Controller().press(pynput.keyboard.Key.esc)  # Trigger 'esc' input
    # keyboard_thread.join()  # Wait for keyboard_thread to finish
    root.destroy()

root = tk.Tk()
root.title("Audio Recorder")

import atexit
# Register the cleanup function to be called on exit
atexit.register(on_closing)


global info_label
info_label = tk.Label(root, text="Press 'Space' to start recording", padx=10, pady=10)
info_label.pack()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()



# # Start GUI and keyboard listener threads
# gui_thread = threading.Thread(target=gui_thread)
# # keyboard_thread = threading.Thread(target=keyboard_listener_thread)

# gui_thread.start()
# # keyboard_thread.start()


# # Update the label from the queue
# while True:
#     message = info_queue.get()
#     info_label.config(text=message)


func record_audio


In [6]:
# print(space_event_queue.get())

In [7]:
print(len(audio_data))
# print(len(audio_input_queue.get()))
while not audio_input_queue.empty():
    try:
        audio = audio_input_queue.get(block=False)
        print(len(audio))
    except:
        continue
    audio_input_queue.task_done()

0


In [1]:
import torchaudio
from io import BytesIO
import sounddevice as sd

In [28]:



# # Play the audio using sounddevice
# def play_audio(wav_file):
#     # load wave file or file-like object
#     data, sr = torchaudio.load(wav_file)
#     # play audio
#     sd.play(data[0].numpy(), sr)
#     # wait until finish playing
#     sd.wait()

In [11]:
print(np.max(audio_data))
result = np.concatenate(audio_data)
print(result)
# print(np.max(result))
# result2 = np.concatenate(audio)
# print(result2)

sd.play(result, samplerate=44100)

# # change result bytes stream to file-like object
# wav_file_bytesIO = BytesIO(result)

# 1. directly play the audio
# play_audio(result)

13185
[ 0  0 -1 ... -1 -1  2]
